In [1]:
import numpy as np
import pickle as pk

import tensorflow.keras
from tensorflow.keras.models import load_model

test_data_path = "test_data.pk"
MODEL_NAME = "model_2.keras"

model = load_model(MODEL_NAME)

2024-04-25 12:23:29.051136: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
test_data = pk.load(open(test_data_path, "rb"))
X_test = test_data["X"]
X_test_oh = test_data["X_oh"]
alphabet_size = test_data["alphabet_size"]

y_test = test_data["y"]
event_mapping = test_data["event_mapping"]

len(X_test), len(X_test_oh), len(y_test)

(227925, 227925, 227925)

In [3]:
sample_idx = 5
test = model.predict(np.array(X_test[sample_idx]))

1/1 [==============================] - 0s 321ms/step


In [4]:
idx = 6
np.argmax(test[idx]), np.argmax(X_test_oh[sample_idx][idx])

(22, 77)

In [5]:
#X_test[5]
y_test[sample_idx]

1

X_pred = list()
for idx, seq in enumerate(X_test):
    current_prediction = None
    for i in range(len(seq)):
        pred = model(np.array(seq[:i+1]))
        if current_prediction is None:
            current_prediction = pred
        else:
            current_prediction = np.vstack((current_prediction, pred))
    X_pred.append(current_prediction)
    break
    if idx % 1000 == 0:
        print(idx)
    if idx == 15000:
        break

In [6]:
import timeit
start_time = timeit.default_timer()

X_pred = list()
for idx, seq in enumerate(X_test):
    current_prediction = model(np.array(seq))
    #for i in range(len(seq)):
        #pred = model(np.array(seq[:i+1]))
        #pred = model(np.array(seq))
        #print("Predict on: ", seq[:i+1].shape)
        #print("Shape of prediction: ", pred.shape)
        #if current_prediction is None:
        #    current_prediction = pred
        #else:
        #    current_prediction = np.vstack((current_prediction, pred))
        #print("Current_prediction: ", current_prediction.shape)
    X_pred.append(current_prediction)
    if idx % 5000 == 0:
        print(idx)
    #if idx == 15000: # TODD: do more here
    #    break

elapsed_time = timeit.default_timer() - start_time
print(elapsed_time)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
1652.9384575660006


In [7]:
X_test[0], X_pred[0].shape, X_test_oh[0].shape

([79, 61, 61, 89, 89, 61, 11, 11, 11, 11, 11], TensorShape([11, 92]), (11, 92))

In [8]:
print(len(X_pred))

227925


In [9]:
import pickle as pk

pk.dump(X_pred, open("X_pred.pk", "wb"))